In [144]:
# Check if running in Google Colab
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    # Define the base path for your data directory for Colab
    basePath = '/content/drive/MyDrive'

else:
    # Running locally or in a different environment
    basePath = '..'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

In [146]:
df = pd.read_csv(f'{basePath}/data/processed/grouped_data2.csv')
only_req_cols = ['COUNTRY', 'DATE', 'TAVG', 'TMIN', 'TMAX', 'PRCP']
trimmed_df = df[only_req_cols].to_csv(f'{basePath}/data/processed/trimmedData.csv')

###**Essential Fns**

In [147]:
def filter_and_select_data(df, column_name, start_year):
    """Selects columns and filters data from a starting year."""
    selected_columns = ['COUNTRY', 'DATE', column_name]
    df_filtered = df[selected_columns].copy()
    return df_filtered[df_filtered['DATE'] >= start_year]

In [148]:
def get_sufficient_countries(df, column_name, completeness_threshold):
    """Returns a list of countries with data completeness >= a given threshold."""
    completeness = df.groupby('COUNTRY')[column_name].apply(lambda x: x.count() / len(x))
    return completeness[completeness >= completeness_threshold].index.tolist()

In [149]:
def split_data_by_year(df, train_end_year):
    """Splits a DataFrame into training and testing sets based on year."""
    df_train = df[df['DATE'] <= train_end_year].copy()
    df_test = df[df['DATE'] > train_end_year].copy()
    return df_train, df_test

In [150]:
def handle_missing_values(df, column_name):
    """Fills missing values in a DataFrame column using a sequence of
    interpolation, backfill, and forward fill, grouped by country.
    """
    df[column_name] = df.groupby('COUNTRY')[column_name].transform(lambda x: x.interpolate())
    df[column_name] = df.groupby('COUNTRY')[column_name].transform(lambda x: x.bfill())
    df[column_name] = df.groupby('COUNTRY')[column_name].transform(lambda x: x.ffill())
    return df

In [151]:
def ensure_consecutive_years(df, column_name, start_year, end_year):
    """
    Ensures a DataFrame has a complete time series of years for each country
    up to a specified end_year. Fills in missing years and interpolates data.
    """
    processed_dfs = []
    unique_countries = df['COUNTRY'].unique()
    for country in unique_countries:
        country_data = df[df['COUNTRY'] == country]

        all_years = list(range(start_year, end_year + 1))
        country_years_df = pd.DataFrame({'COUNTRY': country, 'DATE': all_years})

        merged_country_df = pd.merge(country_years_df, country_data, on=['COUNTRY', 'DATE'], how='left')
        merged_country_df[column_name] = merged_country_df[column_name].interpolate(limit_direction='both')
        processed_dfs.append(merged_country_df)

    return pd.concat(processed_dfs, ignore_index=True)

In [ ]:
def filter_by_common_countries(df_train, df_test, column_name):
    """
    Finds countries with at least one non-missing value in both training and
    testing DataFrames and filters both to include only those countries.

    Returns:
        tuple: A tuple containing the filtered training and testing DataFrames.
    """
    countries_with_train_data = df_train.dropna(subset=[column_name])['COUNTRY'].unique()

    countries_with_test_data = df_test.dropna(subset=[column_name])['COUNTRY'].unique()

    common_countries = set(countries_with_train_data).intersection(set(countries_with_test_data))

    df_train_filtered = df_train[df_train['COUNTRY'].isin(common_countries)].copy()
    df_test_filtered = df_test[df_test['COUNTRY'].isin(common_countries)].copy()

    return df_train_filtered, df_test_filtered

In [ ]:
def prepare_column_data_pipeline(df, column_name, start_year=1980, train_end_year=2010, completeness_threshold=0.8):
    """
    Main pipeline function to prepare and clean time series data for a specific column.
    """
    df_filtered = filter_and_select_data(df, column_name, start_year)

    sufficient_countries = get_sufficient_countries(df_filtered, column_name, completeness_threshold)
    df_filtered = df_filtered[df_filtered['COUNTRY'].isin(sufficient_countries)].copy()

    df_train, df_test = split_data_by_year(df_filtered, train_end_year)

    #Adding just to check::::
    df_train = ensure_consecutive_years(df_train, column_name, start_year = start_year, end_year = train_end_year)
    df_test = ensure_consecutive_years(df_test, column_name, train_end_year + 1, 2025)

    df_train = handle_missing_values(df_train, column_name)
    df_test = handle_missing_values(df_test, column_name)

    df_train, df_test = filter_by_common_countries(df_train, df_test, column_name)

    df_train_completed = ensure_consecutive_years(df_train, column_name, start_year=start_year, end_year=train_end_year)
    df_test_completed = ensure_consecutive_years(df_test, column_name, train_end_year + 1, 2025)

    return df_train_completed, df_test_completed

###**Training Stuff**

In [ ]:
def find_best_arima_order(time_series, arima_orders):
    """
    Finds the best ARIMA model order for a single time series based on AIC.

    Args:
        time_series (pd.Series): The time series data for a single country.
        arima_orders (list of tuples): A list of (p, d, q) orders to test.

    Returns:
        tuple: A tuple containing the best model fit object and its order.
               Returns (None, None) if no model can be fitted.
    """
    best_aic = np.inf
    best_order = None
    best_model_fit = None

    for order in arima_orders:
        try:
            model = ARIMA(time_series, order=order, freq='YS')
            model_fit = model.fit()

            # Get the AIC
            aic = model_fit.aic

            # to Update best AIC and order if current model is better
            if aic < best_aic:
                best_aic = aic
                best_order = order
                best_model_fit = model_fit
                print(f"  Order: {order}, AIC: {aic}")

        except Exception as e:
            continue

    return best_model_fit, best_order

In [ ]:
def train_models_for_countries(df, column_name, arima_orders):
    """
    Trains the best ARIMA model for each country in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame containing country data.
        column_name (str): The column containing the time series data.
        arima_orders (list of tuples): A list of (p, d, q) orders to test.

    Returns:
        dict: A dictionary where keys are country names and values are dictionaries
              containing the 'best_model' object and its 'best_order' and 'best_aic'.
    """
    best_arima_models = {}
    unique_countries = df['COUNTRY'].unique()

    for country in unique_countries:
        # Filter data for the current country
        country_data = df[df['COUNTRY'] == country]

        # Set the 'DATE' column as the index and convert it to a DatetimeIndex
        time_series = country_data.set_index(pd.to_datetime(country_data['DATE'], format='%Y'))[column_name]
        
        best_model, best_order = find_best_arima_order(time_series, arima_orders)

        # Store the best model and its details
        if best_model is not None:
            best_arima_models[country] = {
                'best_model': best_model,
                'best_order': best_order,
                'best_aic': best_model.aic
            }
        else:
            print(f"Warning: No best model found for {country}. Skipping.")

    return best_arima_models

In [156]:
# def sanitize_country_name(country):
#     # Replace spaces with underscores and remove brackets and other problematic chars
#     return country.replace(' ', '_')

In [ ]:
def save_trained_models(models_dict, models_dir):
    """
    Saves a dictionary of trained models to individual files in a specified directory.

    #Most probably:
    Args:
        models_dict (dict): A dictionary of trained model objects. Keys are country names.
                            Values should contain the model object under the 'best_model' key.
        models_dir (str): The directory path where the models should be saved.
    """

    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    for country, model_info in models_dict.items():
        best_model = model_info.get('best_model')

        if best_model is not None:

            sanitized_country_name = country
            filename = f"{sanitized_country_name}_arima_model.joblib"
            filepath = os.path.join(models_dir, filename)

            joblib.dump(best_model, filepath)
        else:
            print(f"Warning: No model found for {country}, skipping save.")

###**Prediction and Errors**

In [ ]:
def load_trained_models(models_dir):
    """
    Loads all trained ARIMA model objects from a specified directory.
    """
    loaded_models = {}

    if not os.path.exists(models_dir):
        print(f"Directory not found: {models_dir}")
        return loaded_models

    for filename in os.listdir(models_dir):
        if filename.endswith('_arima_model.joblib'):
            filepath = os.path.join(models_dir, filename)
            try:
                sanitized_country_name = filename.replace('_arima_model.joblib', '')

                model = joblib.load(filepath)
                loaded_models[sanitized_country_name] = model
            except Exception as e:
                print(f"Warning: Could not load model from {filepath}. Error: {e}")

    if not loaded_models:
        print(f"No models found in directory: {models_dir}")
    else:
        print(f"Successfully loaded {len(loaded_models)} models.")

    return loaded_models

In [ ]:
def predict_single_country(model_fit, n_forecast_steps: int = 10):
    """
    Makes a prediction for a specified number of future steps using a trained ARIMA model.

    Args:
        model_fit (SARIMAXResultsWrapper): The fitted ARIMA model object.
        n_forecast_steps (int): The number of time steps (e.g., years) to forecast into the future.

    Returns:
        pd.Series: A pandas Series containing the forecasted values.
                   The index of the Series will be the corresponding years.
                   Returns None if an error occurs.
    """
    try:
        forecast_result = model_fit.get_forecast(steps=n_forecast_steps)

        predictions = forecast_result.predicted_mean

        return predictions

    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return None

In [160]:
def evaluate_single_country(actual_data, predictions):
    """
    Calculates key error metrics for a single country's predictions.

    Args:
        actual_data (pd.Series): The actual time series data.
        predictions (pd.Series): The predicted time series data.

    Returns:
        dict: A dictionary of calculated error metrics (MSE, RMSE, MAE).
    """
    mse = mean_squared_error(actual_data, predictions)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(actual_data, predictions)

    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae}

In [ ]:
def predict_and_evaluate_all_models(models_dir, df_test, column_name):
    """
    Orchestrates the prediction and evaluation process for all countries.

    Args:
        models_dir (str): The directory containing the saved trained models.
        df_test (pd.DataFrame): The cleaned DataFrame with testing data.
        column_name (str): The name of the column to predict.

    Returns:
        dict: A dictionary where keys are country names and values are their
              calculated error metrics.
    """
    all_country_errors = {}

    loaded_models = load_trained_models(models_dir)

    if not loaded_models:
        print("No models were loaded. Cannot proceed with prediction.")
        return all_country_errors

    print("\nStarting prediction and evaluation...")

    # Loop through each country for which we have a trained model
    for country, model in loaded_models.items():

        country_test_data = df_test[df_test['COUNTRY'] == country].copy()

        # Ensure the test data has a proper DatetimeIndex for alignment
        country_test_data.set_index(pd.to_datetime(country_test_data['DATE'], format='%Y'), inplace=True)
        actual_data = country_test_data[column_name]

        # Get the start and end years for prediction from the actual data
        start_year = actual_data.index.min().year
        end_year = actual_data.index.max().year

        try:
            predictions = predict_single_country(model, n_forecast_steps=15)

            print(f"INSIDE \n\n\n predict_and_evaluate_fn")
            print(f"PReditctions for {country} is {predictions}")

            errors = evaluate_single_country(actual_data, predictions)

            all_country_errors[country] = errors
            print(f"Processed {country}: RMSE = {errors['RMSE']:.2f}, MAE = {errors['MAE']:.2f}")

        except Exception as e:
            print(f"Error processing {country}: {e}")

    print("\nPrediction and evaluation complete.")
    return all_country_errors

###**Combining data for more training**

In [162]:
def combine_data_for_final_training(df_train, df_test):
    """
    Combines the cleaned training and testing DataFrames into a single DataFrame,
    and sorts the result for clean time series analysis.

    Args:
        df_train (pd.DataFrame): The cleaned DataFrame with training data.
        df_test (pd.DataFrame): The cleaned DataFrame with testing data.

    Returns:
        pd.DataFrame: A single DataFrame containing data from both periods,
                      sorted by country and date.
    """
    df_combined = pd.concat([df_train, df_test], ignore_index=True)

    # Sort the data by country and then by date for a clean, final product
    df_combined_sorted = df_combined.sort_values(by=['COUNTRY', 'DATE']).reset_index(drop=True)

    df_combined_sorted.set_index(pd.to_datetime(df_combined_sorted['DATE'], format='%Y'), inplace=True)

    return df_combined_sorted

In [163]:
def train_final_models(df_combined, column_name, arima_orders):
    """
    Trains the final ARIMA models on the complete dataset for each country.

    This function reuses our verified training function.

    Args:
        df_combined (pd.DataFrame): The DataFrame containing the complete time series data.
        column_name (str): The column containing the time series data.
        arima_orders (list of tuples): A list of (p, d, q) orders to test.

    Returns:
        dict: A dictionary of final trained models for each country.
    """
    print(f"Starting final model training for column: {column_name}")
    final_models = train_models_for_countries(df_combined, column_name, arima_orders)
    print("Final model training complete.")
    return final_models

In [ ]:
def make_predictions(models_dir, df_combined, column_name, number_of_years: int = 10):
    """
    Args:
        models_dir (str): The directory containing the saved trained models. For our case it will be the same as above as
        it overwrites when training on the large dataset..
        .....
        .....
    """
    all_predicted_dfs = {}

    loaded_models = load_trained_models(models_dir)

    if not loaded_models:
        print("No models were loaded. Cannot proceed with predictions.")
        return all_predicted_dfs

    print(f"\nStarting Predictions")

    for country, model in loaded_models.items():

        try:
            # ake predictions
            predictions = predict_single_country(model, n_forecast_steps=number_of_years)

            print(f"INSIDE \n\n\n predict fn")
            print(f"PReditctions for {country} is {predictions}")

            if predictions is not None:
                all_predicted_dfs[country] = predictions

        except Exception as e:
            print(f"Error processing {country}: {e}")

    print("\nPrediction complete.")
    return all_predicted_dfs

In [ ]:
###Testing Till this::
filepath = f'{basePath}/data/processed/trimmedData.csv'
df = pd.read_csv(filepath)
column_name = 'TAVG'
models_dir = f'{basePath}/modelsNew/trainedModels'
arima_orders = [(1, 1, 2), (2, 1, 0), (2, 1, 1), (2, 1, 2), (3, 1, 0), (3, 1, 1), (3, 1, 2), (4, 1, 0), (4, 1, 1), (4, 1, 2), (5, 1, 0), (5, 1, 1), (5, 1, 2)]
df_train, df_test = prepare_column_data_pipeline(df, column_name, start_year=1980, train_end_year=2010, completeness_threshold=0.8)
best_models_dict = train_models_for_countries(df_train, column_name, arima_orders)
save_trained_models(best_models_dict, models_dir)

loaded_models = load_trained_models(models_dir)


# 4. Predict and evaluate the models
evaluation_results = predict_and_evaluate_all_models(models_dir, df_test, column_name)
print("Evaluation Results:", evaluation_results)

In [ ]:
df_combined = combine_data_for_final_training(df_train, df_test)
models_dir = f'{basePath}/modelsNew/trainedModels'
column_name = 'TAVG'
latest_models_dict = train_models_for_countries(df_combined, column_name, arima_orders)
save_trained_models(latest_models_dict, models_dir)
predictions = make_predictions(models_dir, df_combined, column_name, number_of_years=10)
predictions

In [ ]:
df_combined = combine_data_for_final_training(df_train, df_test)
models_dir = f'{basePath}/modelsNew/trainedModels'
column_name = 'TAVG'
latest_models_dict = train_final_models(df_combined, column_name, arima_orders)
save_trained_models(latest_models_dict, models_dir)

# Assign the result of make_predictions to a new variable
country_predictions = make_predictions(models_dir, df_combined, column_name, number_of_years=10)

df_final_pred = []
# Iterate over the new variable containing the predictions dictionary
for country, predictions in country_predictions.items():
    # Add a check to ensure 'predictions' is a pandas Series
    if isinstance(predictions, pd.Series):
        # Repeat the country name for each prediction
        df_curr = pd.DataFrame({'COUNTRY': [country] * len(predictions), 'DATE': predictions.index, column_name: predictions.values})
        df_final_pred.append(df_curr)
    else:
        print(f"Skipping {country}: Predictions are not a pandas Series.")


df_final_pred = pd.concat(df_final_pred, ignore_index=True)
df_final_pred

In [168]:
df_final_pred.to_csv(f'{basePath}/data/processed/finalPredictions.csv', index=False)

In [169]:
df_final_pred['COUNTRY'].nunique()

36

In [170]:
# Ensure 'DATE' column is datetime objects in both dataframes
df_combined['DATE'] = pd.to_datetime(df_combined.index) # DATE was the index in df_combined
df_final_pred['DATE'] = pd.to_datetime(df_final_pred['DATE'])

# Combine the historical and predicted dataframes
df_combined_with_predictions = pd.concat([df_combined.reset_index(drop=True), df_final_pred], ignore_index=True)

# Sort the combined dataframe by country and date
df_combined_with_predictions_sorted = df_combined_with_predictions.sort_values(by=['COUNTRY', 'DATE']).reset_index(drop=True)

In [171]:
# Define the path to save the combined data
output_path_combined = f'{basePath}/data/processed/combined_predictions_sorted.csv'

# Save the combined and sorted dataframe to a CSV file
df_combined_with_predictions_sorted.to_csv(output_path_combined, index=False)

###**Top n Riskiest countries + Plotting**

In [ ]:
def calculate_percentage_change_and_get_top_n(df, column_name, start1, end1, start2, end2, top_n=10):
    """
    Calculates the percentage change of a column between two periods for each country
    and returns the top N countries with the largest percentage change.

    Args:
        df (pd.DataFrame): The input DataFrame containing country data.
        column_name (str): The column to calculate the percentage change for.
        start1 (int): The start year of the first period.
        end1 (int): The end year of the first period.
        start2 (int): The start year of the second period.
        end2 (int): The end year of the second period.
        top_n (int): The number of top countries to return.

    Returns:
        pd.DataFrame: A DataFrame containing the top N countries and their
                      calculated percentage change, sorted by percentage change.
    """
    # Ensure 'DATE' is in datetime format for filtering
    df['DATE'] = pd.to_datetime(df['DATE'])

    # Filter data for the two periods
    df_period1 = df[(df['DATE'].dt.year >= start1) & (df['DATE'].dt.year <= end1)].copy()
    df_period2 = df[(df['DATE'].dt.year >= start2) & (df['DATE'].dt.year <= end2)].copy()

    # Calculate the average of the column for each country in each period
    avg_period1 = df_period1.groupby('COUNTRY')[column_name].mean()
    avg_period2 = df_period2.groupby('COUNTRY')[column_name].mean()

    # Combine the averages into a single DataFrame
    combined_avg = pd.DataFrame({
        'Avg_Period1': avg_period1,
        'Avg_Period2': avg_period2
    })

    # Calculate the percentage change
    combined_avg['Percentage_Change'] = ((combined_avg['Avg_Period2'] - combined_avg['Avg_Period1']) / combined_avg['Avg_Period1']) * 100
    combined_avg['Percentage_Change'] = combined_avg['Percentage_Change'].replace([np.inf, -np.inf], np.nan)


    # Drop rows where percentage change could not be calculated (e.g., due to missing data)
    combined_avg.dropna(subset=['Percentage_Change'], inplace=True)

    # Sort by percentage change in descending order and get the top N
    top_n_countries = combined_avg.sort_values(by='Percentage_Change', ascending=False).head(top_n)

    return top_n_countries.reset_index()

In [ ]:
start_year_1 = 2015
end_year_1 = 2025
start_year_2 = 2026
end_year_2 = 2035
column_to_analyze = 'TAVG'
num_top_countries = 10

top_riskiest_countries = calculate_percentage_change_and_get_top_n(
    df_combined_with_predictions_sorted.copy(), 
    column_to_analyze,
    start_year_1,
    end_year_1,
    start_year_2,
    end_year_2,
    top_n=num_top_countries
)

print(f"Top {num_top_countries} countries with the largest percentage change in {column_to_analyze}:")
display(top_riskiest_countries)

Top 10 countries with the largest percentage change in TAVG:


,COUNTRY,Avg_Period1,Avg_Period2,Percentage_Change
0,Switzerland,0.654545,1.401069,114.052193
1,Greenland [Denmark],-7.433333,-9.763399,31.346180
2,China,14.379419,18.000000,25.178906
3,Svalbard [Norway],-2.227273,-2.568371,15.314628
4,Germany,10.543182,11.620544,10.218569
5,Russia,-1.078464,-1.185539,9.928522
6,Sweden,5.715152,6.250538,9.367842
7,Estonia,7.304545,7.911645,8.311251
8,Austria,6.642424,7.133333,7.390511
9,Finland,3.754545,3.906888,4.057547


In [174]:
def plot_temperature_trends_two_periods(df, column_name, start1, end1, start2, end2, countries_to_plot=None):
    """
    Generates a line plot of the specified column for given countries over two time periods.

    Args:
        df (pd.DataFrame): The input DataFrame containing country data.
        column_name (str): The column to plot.
        start1 (int): The start year of the first period.
        end1 (int): The end year of the first period.
        start2 (int): The start year of the second period.
        end2 (int): The end year of the second period.
        countries_to_plot (list, optional): A list of country names to plot.
                                            If None, all countries in the filtered data will be plotted.
    """
    # Ensure 'DATE' is in datetime format and set as index for easier plotting
    df['DATE'] = pd.to_datetime(df['DATE'])
    df_indexed = df.set_index('DATE')

    # Filter data for the two periods
    df_period1 = df_indexed[(df_indexed.index.year >= start1) & (df_indexed.index.year <= end1)].copy()
    df_period2 = df_indexed[(df_indexed.index.year >= start2) & (df_indexed.index.year <= end2)].copy()

    # Combine the data from the two periods for plotting
    df_combined_periods = pd.concat([df_period1, df_period2])

    if countries_to_plot is not None:
        df_combined_periods = df_combined_periods[df_combined_periods['COUNTRY'].isin(countries_to_plot)]

    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df_combined_periods, x=df_combined_periods.index, y=column_name, hue='COUNTRY')
    plt.title(f'{column_name} Trend Over Time for Selected Countries ({start1}-{end1} and {start2}-{end2})')
    plt.xlabel('Year')
    plt.ylabel(column_name)
    plt.grid(True)
    plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
# Example usage with the top 10 riskiest countries:
start_year_1 = 2000
end_year_1 = 2025
start_year_2 = 2026
end_year_2 = 2035
column_to_plot = 'TAVG'
top_countries_list = top_riskiest_countries['COUNTRY'].tolist() 

plot_temperature_trends_two_periods(
    df_combined_with_predictions_sorted.copy(), # Use the combined dataframe
    column_to_plot,
    start_year_1,
    end_year_1,
    start_year_2,
    end_year_2,
    countries_to_plot=top_countries_list
)

In [176]:
def train_predict_save_column(df_combined, models_dir, column_name, arima_orders, number_of_years=10):
    """
    Trains ARIMA models for each country on a specified column, makes predictions,
    formats the predictions, and saves them to a CSV file.

    Args:
        df_combined (pd.DataFrame): The input DataFrame containing combined data.
        models_dir (str): The directory to save the trained models.
        column_name (str): The column to train models and make predictions for.
        arima_orders (dict): A dictionary where keys are country names and values
                             are the ARIMA orders (p, d, q).
        number_of_years (int): The number of years to predict.

    Returns:
        pd.DataFrame: A DataFrame containing the formatted predictions.
    """
    print(f"Starting final model training for column: {column_name}")
    latest_models_dict = train_final_models(df_combined, column_name, arima_orders)
    save_trained_models(latest_models_dict, models_dir)

    print("\nStarting Predictions")
    country_predictions = make_predictions(models_dir, df_combined, column_name, number_of_years=number_of_years)

    df_final_pred = []
    for country, predictions in country_predictions.items():
        if isinstance(predictions, pd.Series):
            df_curr = pd.DataFrame({'COUNTRY': [country] * len(predictions), 'DATE': predictions.index, column_name: predictions.values})
            df_final_pred.append(df_curr)
        else:
            print(f"Skipping {country}: Predictions are not a pandas Series.")

    df_final_pred = pd.concat(df_final_pred, ignore_index=True)

    # Save the predictions to a CSV file
    output_filename = f'finalPredictions_{column_name}.csv'
    output_path = os.path.join(f'{basePath}/data/processed', output_filename)
    df_final_pred.to_csv(output_path, index=False)
    print(f"\nPredictions for column '{column_name}' saved to {output_path}")

    return df_final_pred

In [ ]:
# Call the new function with the desired column name
column_name_for_prediction = 'TAVG' # Or any other column name you want to use
df_tavg_predictions = train_predict_save_column(df_combined, models_dir, column_name_for_prediction, arima_orders, number_of_years=10)

# You can now work with df_tavg_predictions if needed, for example, display it
display(df_tavg_predictions.head())